In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import PIL,cv2
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.applications import EfficientNetB4,ResNet50,EfficientNetB5
from sklearn.model_selection import train_test_split

SEED=48
DEBUG=False

os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
# gpus = tf.config.experimental.list_physical_devices('GPU') 
# for gpu in gpus: 
#     tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
BASE_PATH='../input/shopee-product-matching/'
train=pd.read_csv(BASE_PATH+"train.csv")
train['image_path']=BASE_PATH+'train_images/'+train.image

In [3]:
id_to_label_mapping=dict(zip(train.label_group.unique(),range(train.label_group.nunique())))
train["label_number"]=train.label_group.map(id_to_label_mapping)

NUM_CLASSES=train.label_group.nunique()
HEIGHT,WIDTH=128,128
CHANNELS=3
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE=40

print("Sample of Available Data")
print(train.head(),'\n')

Sample of Available Data
         posting_id                                 image       image_phash  \
0   train_129225211  0000a68812bc7e98c42888dfb1c07da0.jpg  94974f937d4c2433   
1  train_3386243561  00039780dfc94d01db8676fe789ecd05.jpg  af3f9460c2838f0f   
2  train_2288590299  000a190fdd715a2a36faed16e2c65df7.jpg  b94cb00ed3e50f78   
3  train_2406599165  00117e4fc239b1b641ff08340b429633.jpg  8514fc58eafea283   
4  train_3369186413  00136d1cf4edede0203f32f05f660588.jpg  a6f319f924ad708c   

                                               title  label_group  \
0                          Paper Bag Victoria Secret    249114794   
1  Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...   2937985045   
2        Maling TTS Canned Pork Luncheon Meat 397 gr   2395904891   
3  Daster Batik Lengan pendek - Motif Acak / Camp...   4093212188   
4                  Nescafe \xc3\x89clair Latte 220ml   3648931069   

                                          image_path  label_number  
0  ../input/shop

In [4]:
# Data processing function for creating tf.data dataset 
# Converting image path dataset to image label dataset
def process_data(image_path,label):
    img=tf.io.read_file(image_path)
    img=tf.image.decode_jpeg(img,channels=CHANNELS)
    img=tf.image.resize(img,[HEIGHT,WIDTH])
    return img,label

# function to improve dataset processing speed 
def configure_for_performance(ds,batch_size):
    ds=ds.cache('/kaggle/dump.tfcache')
    
    ds=ds.shuffle(buffer_size=1024)
    ds=ds.batch(BATCH_SIZE)
    ds=ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [5]:
x_train,x_valid=train_test_split(train,test_size=0.1,random_state=SEED,shuffle=True)

# image path & label dataset
train_ds=tf.data.Dataset.from_tensor_slices((x_train.image_path.values,x_train.label_number.values))
valid_ds=tf.data.Dataset.from_tensor_slices((x_valid.image_path.values,x_valid.label_number.values))

# image & label dataset
train_ds=train_ds.map(process_data,num_parallel_calls=AUTOTUNE)
valid_ds=valid_ds.map(process_data,num_parallel_calls=AUTOTUNE)

# improving dataset by shuffling dataset,creating image batch and prefetching dataset
# more information : https://www.tensorflow.org/guide/data_performance
# Batch image and label dataset
train_ds_batch = configure_for_performance(train_ds, 8)
valid_ds_batch = valid_ds.batch(8)

print("Dataset before reconfiguring")
print("train data:",train_ds.cardinality())
print("valid data",valid_ds.cardinality(),'\n')

print("Dataset after reconfiguring(BATCH+SHUFFLE+PREFETCH)")
print("train data:",train_ds_batch.cardinality())
print("valid data",valid_ds_batch.cardinality(),'\n')

2022-06-05 13:47:22.511688: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 13:47:22.620070: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 13:47:22.620968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Dataset before reconfiguring
train data: tf.Tensor(30825, shape=(), dtype=int64)
valid data tf.Tensor(3425, shape=(), dtype=int64) 

Dataset after reconfiguring(BATCH+SHUFFLE+PREFETCH)
train data: tf.Tensor(771, shape=(), dtype=int64)
valid data tf.Tensor(429, shape=(), dtype=int64) 



2022-06-05 13:47:22.624265: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-05 13:47:22.625501: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 13:47:22.626438: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 13:47:22.627244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [6]:
# Function to get our f1 score
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

In [7]:
def pairwise_distances(embeddings):
    dot_product = tf.linalg.matmul(embeddings, tf.transpose(embeddings))
    square_norm = tf.linalg.diag_part(dot_product)
    distances = tf.expand_dims(square_norm, 1) - 2.0 * dot_product + tf.expand_dims(square_norm, 0)
    distances = tf.math.maximum(distances, 0.0)

    mask = tf.cast(tf.equal(distances, 0.0),tf.float32)
    distances = distances + mask * 1e-16
    distances = tf.math.sqrt(distances)
    distances = distances * (1.0 - mask)

    return distances

In [8]:
def get_anchor_positive_triplet_mask(labels):
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.math.logical_not(indices_equal)

    labels_equal = tf.math.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    mask = tf.math.logical_and(indices_not_equal, labels_equal)

    return mask

In [9]:
def get_anchor_negative_triplet_mask(labels):
    labels_equal = tf.math.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    mask = tf.math.logical_not(labels_equal)

    return mask

In [10]:
def get_triplet_mask(labels):
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.math.logical_not(indices_equal)
    i_not_equal_j = tf.expand_dims(indices_not_equal, 2)
    i_not_equal_k = tf.expand_dims(indices_not_equal, 1)
    j_not_equal_k = tf.expand_dims(indices_not_equal, 0)

    distinct_indices = tf.math.logical_and(tf.math.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)


    label_equal = tf.math.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    i_equal_j = tf.expand_dims(label_equal, 2)
    i_equal_k = tf.expand_dims(label_equal, 1)

    valid_labels = tf.math.logical_and(i_equal_j, tf.logical_not(i_equal_k))

    mask = tf.math.logical_and(distinct_indices, valid_labels)

    return mask

In [11]:
class TripletLossFn(tf.keras.losses.Loss):
    def __init__(self,margin=1.0,**kwargs):
        super().__init__(**kwargs)
        self.margin = margin
  
    def call(self,y_true,y_pred):

        labels = tf.convert_to_tensor(y_true)
        labels = tf.squeeze(labels,axis=-1)
        embeddings = tf.convert_to_tensor(y_pred)

        pairwise_dist = pairwise_distances(embeddings)

        mask_anchor_positive = get_anchor_positive_triplet_mask(labels)
        mask_anchor_positive = tf.cast(mask_anchor_positive,tf.float32)

        anchor_positive_dist = tf.math.multiply(mask_anchor_positive, pairwise_dist)

        hardest_positive_dist = tf.math.reduce_max(anchor_positive_dist, axis=1, keepdims=True)


        mask_anchor_negative = get_anchor_negative_triplet_mask(labels)
        mask_anchor_negative = tf.cast(mask_anchor_negative,tf.float32)

        max_anchor_negative_dist = tf.math.reduce_max(pairwise_dist, axis=1, keepdims=True)
        anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)


        hardest_negative_dist = tf.math.reduce_min(anchor_negative_dist, axis=1, keepdims=True)
    

        triplet_loss = tf.math.maximum(hardest_positive_dist - hardest_negative_dist + self.margin, 0.0)

        triplet_loss = tf.math.reduce_mean(triplet_loss)

        return triplet_loss
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config,"margin":self.margin}

In [12]:
# Model creation using a pretrained model
def create_model(pretrained_model):  
    
    model=tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ])
    
    return model

# funtion to compile a choosen model
def compile_model(model,LR=0.001):
    
    optimizer = tf.keras.optimizers.Adam(lr=LR)
    
    loss=TripletLossFn(0.7)
    
    metrics = [
       tf.keras.metrics.SparseCategoricalAccuracy()
    ]

    model.compile(optimizer=optimizer, loss=loss)
    
    return model

# Callback list generation
def callback_creation(model_path):
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        mode='min',
        factor=0.1,
        patience=3,
        verbose=0
    )
    
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=model_path,
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=1,
    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        patience=10, 
        verbose=1
    )
    
    callbacks=[reduce_lr,model_checkpoint,early_stopping]
    
    return callbacks

# model training
def model_training(pretrained_model,model_path):
    EPOCH_COUNTS=1
    VERBOSE=1
    LR=0.0001

    tf.keras.backend.clear_session();

    model=create_model(pretrained_model)
    model=compile_model(model,LR=LR)
    callback_list=callback_creation(model_path)


    history=model.fit(
                        train_ds_batch,
                        validation_data=valid_ds_batch,
                        epochs=EPOCH_COUNTS,
                        callbacks=callback_list,
                    )

In [13]:
efnb4_model_path='./best_model_efnb4.h5'
pretrained_model=EfficientNetB4(include_top=False, weights='imagenet',input_shape=[HEIGHT,WIDTH, 3])
model_training(pretrained_model,efnb4_model_path)

71696384/71686520 [==============================] - 0s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-06-05 13:47:42.606593: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-05 13:47:52.930423: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


745/771 [===========================>..] - ETA: 6s - loss: 0.0035

2022-06-05 13:51:09.356627: W tensorflow/core/kernels/data/cache_dataset_ops.cc:233] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


771/771 [==============================] - 244s 279ms/step - loss: 0.0034 - val_loss: 6.7838e-04

Epoch 00001: val_loss improved from inf to 0.00068, saving model to ./best_model_efnb4.h5


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
